# Implementation of open VLLMS

## -- Install the required Libraries if needed--

In [2]:
pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 10.3 MB/s eta 0:00:0000:0100:01


## 1-Import Required Libraries 

In [ ]:
import os
import torch
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

## 2- Load the Pre-trained Model and proces

In [4]:
model_id = "Qwen/Qwen2-VL-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

processor = Qwen2VLProcessor.from_pretrained(model_id)

`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

## 3-Generate Function

In [7]:
def generate_answer(image_path, query, max_new_tokens=256):
    image = Image.open(image_path).convert("RGB")
    sample = {
        "messages": [
            {"role": "system", "content": [{"type": "text", "text": "You are a Vision Language Model. Answer concisely."}]},
            {"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": query}]}
        ]
    }

    text_input = processor.apply_chat_template(sample['messages'][1:2], tokenize=False, add_generation_prompt=True)
    image_inputs, _ = process_vision_info(sample['messages'])

    model_inputs = processor(text=[text_input], images=image_inputs, return_tensors="pt").to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=max_new_tokens)

    trimmed_generated_ids = [out_ids[len(in_ids):] for in_ids, out_ids in zip(model_inputs.input_ids, generated_ids)]

    output_text = processor.batch_decode(trimmed_generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    return output_text[0]

## 4-Running the Function

In [22]:
import os

# current working directory
cwd = os.getcwd()
print("Current working directory:", cwd)
image_path = "Flag_of_Cameroon.svg.png"
# absolute path for the image (uses existing `image_path` variable)
abs_image_path = os.path.abspath(image_path)
print("Absolute image path:", abs_image_path)

# check if the image exists
print("Image exists:", os.path.exists(abs_image_path))

Current working directory: /content
Absolute image path: /content/Flag_of_Cameroon.svg.png
Image exists: False


In [ ]:
query = "Describe this Image"
answer = generate_answer(image_path, query)
print("Generated Answer:", answer)

# THANK YOU 